In [3]:
import lightkurve as lk
import pandas as pd
import warnings

file = pd.read_csv('/../NAOS/Bstars_v3/TIC_377Bstars.csv', sep=';')
id_list = file.TIC

tic_list = []
sector_list = []

for i in range(len(id_list)):
    search = lk.search_lightcurve(target=f'TIC {id_list[i]}', author='SPOC', exptime=120)
    for j in range(len(search)):
        sector = search.table['mission'][j][12:]
        target = search.table['target_name'][j]
        tic_list.append(f'{target}'.zfill(10))
        sector_list.append(sector)
        
df_target = pd.DataFrame(tic_list, columns=['TIC'])       
df_sector = pd.DataFrame(sector_list, columns=['Sector'])
main_list = pd.concat([df_target, df_sector], axis=1)
main_df = main_list.groupby('TIC').agg(lambda x: list(set(x))).reset_index()
N_sector = main_list.groupby('TIC').count().reset_index()
final_df = pd.concat([main_df, N_sector.Sector], axis=1)
final_df = final_df.set_axis(['TIC', 'Sectors', 'N_sectors'], axis=1)
for i in range(len(final_df)):
    final_df.Sectors[i] = sorted(final_df.Sectors[i])
    warnings.filterwarnings("ignore")

final_df.to_csv('/../NAOS/Bstars_v3/Count_List_Sectors.csv', sep=';', index=None)